# Week 3. Problem set

Author: Abu Huraira, BS21-SD-02

## Task 1

Implement the following functions over the list of binary digits in Racket using
higher-order functions (`apply`, `map`, `andmap`, `ormap`, `filter`, `foldl`)
and **without explicit recursion**.

(a) Count a given symbol in a list of symbols:

```lisp
(count-symbol 'l '(h e l l o w o r l d)) ==> 3
```

In [45]:
(define (count-symbol symbol lst)
    (foldl (lambda (x y) (if (eq? x symbol) (+ y 1) y)) 0 lst)
)

(display "Testing count-symbol")
(count-symbol 'l '(h e l l o w o r l d))

Testing count-symbol

3

(b) Convert a binary string represented a list of `0`s and `1`s into a (decimal)
number:

```lisp
(binary-to-decimal '(1 0 1 1 0)) ; ==> 22
```

In [46]:
; Let's define our own higher order function `foldl` which gives us the index of the element as well
(define foldl-with-index (lambda (f init lst)
    (define (iter i lst result)
        (if (null? lst)
            result
            (iter (+ i 1) (cdr lst) (f (car lst) i result))
        )
    )
    (iter 0 lst init)
))

; Now we can use `foldl-with-index` to define `binary-to-decimal`
(define (binary-to-decimal bits)
    (foldl-with-index (lambda (bit idx sum) (+ sum (* bit (expt 2 idx)))) 0 (reverse bits))
)


(display "Testing binary-to-decimal\n")
(display (binary-to-decimal '(1 0 1 1 0)))

Testing binary-to-decimal
22

(c) Return the penultimate symbol in a list (you may assume it has enough
symbols):

```lisp
(penultimate '(1 0 0 1 0)) ; ==> 1
(penultimate '(h e l l o)) ; ==> 'l
```

In [47]:
(define (penultimate lst)
  (car (foldl
   (lambda (current result)
     (if (null? result)
         (list current)
         (if (= (length result) 2)
             (cdr result)
             (cons current result))))
   '()
   lst)))


(display "Testing penultimate\n")
(display (penultimate '(1 0 0 1 0)))
(newline)
(display (penultimate '(h e l l o)))


Testing penultimate
1
h

(d) Encode a string by removing leading zeros and replacing each consecutive
substring of digits with its length. For example, `'(0 0 0 1 1 0 1 1 1 0 0)` has
some leading zeros, then 2 ones, then 1 zero, then 3 ones, then 2 zeros, so it
should be encoded as `'(2 1 3 2)`:

```lisp
(encode-with-lengths '(0 0 0 1 1 0 1 1 1 0 0)) ; ==> '(2 1 3 2)
```

In [48]:
(define (strip-zeros lst)
    (cond
      ((eq? (car lst) 0) (strip-zeros (cdr lst)))
      (else lst)
    )
)

(define (list-consecutive-items lst)
  (reverse (foldl (lambda (bit ans)
                    (cond
                      [(null? ans) (cons (list bit) ans)]
                      [(equal? (car (car ans)) bit) (cons (cons bit (car ans)) (cdr ans))]
                      [else (cons (list bit) ans)]))
                  '() lst)))


(define (encode-with-lengths elems)
  (map (lambda (lst) (length lst)) (list-consecutive-items (strip-zeros elems))))

(display "Testing encode-with-lengths")
(encode-with-lengths '(0 0 0 1 1 0 1 1 1 0 0))

Testing encode-with-lengths

'(2 1 3 2)

(e) Decrement a binary number *without converting to decimal*. Decrementing zero
should produce zero:

```lisp
(decrement '(1 0 1 1 0)) ; ==> '(1 0 1 0 1)
(decrement '(1 0 0 0 0)) ; ==> '(1 1 1 1)
(decrement '(0)) ; ==> '(0)
```

In [49]:
(define (decrement binary)
    (cond
        [(and (eq? (car binary) 0) (eq? (length binary) 1) 0)]
        [
            (let* ((carry? #t)
                (decremented (map (lambda (bit)
                                    (if carry?
                                        (cond
                                        ((= bit 1)
                                        (set! carry? #f)
                                        0)
                                        (else 1))
                                        bit))
                                (reverse binary))))
            (reverse (if carry?
                        (cons 1 decremented)
                        decremented)
            ))]))


(display "Testing decrement\n")
(display (decrement '(1 0 1 1 0)))
(newline)
(display (decrement '(1 0 0 0 0)))
(newline)
(display (decrement '(0)))
(newline)

Testing decrement
(1 0 1 0 1)
(0 1 1 1 1)
0


## Task 2

Consider this list where each entry is a tuple of the first name, gender, age,
and last name:

```
(define employees
  '(("John" #:male 29 . "Malkovich") 
    ("Ivan" #:male 18 . "Petrov")
    ("Anna" #:female 22 . "Petrova")
    ("Ivan" #:male 43 . "Ivanov")
    ("Anna" #:female 20 . "Karenina")))
```

(a) Implement a function fullname that takes employee and returns their full
name as a pair of first and last name:

```lisp
(fullname '("John" #:male 29 . "Malkovich"))
; '("John" . "Malkovich")
```

In [50]:
(define employees
  '(("John" #:male 29 . "Malkovich") 
    ("Ivan" #:male 18 . "Petrov")
    ("Anna" #:female 22 . "Petrova")
    ("Ivan" #:male 43 . "Ivanov")
    ("Anna" #:female 20 . "Karenina")))

; fullname: tuple -> pair
(define (fullname tupl)
    (cons (car tupl) (cdddr tupl))
)

; get-all-full-names: list of tuples -> list of pairs
(define (get-all-full-names lst)
    (map fullname lst)
)

(get-all-full-names employees)

'(("John" . "Malkovich") ("Ivan" . "Petrov") ("Anna" . "Petrova") ("Ivan" . "Ivanov") ("Anna" . "Karenina"))

(b) Using higher-order functions (`map`, `ormap`, `andmap`, `filter`, `foldl`) and without explicit recursion, write down an expression that computes a list of entries from employees where each employee is `#:male`.

In [51]:
(define employees
  '(("John" #:male 29 . "Malkovich") 
    ("Ivan" #:male 18 . "Petrov")
    ("Anna" #:female 22 . "Petrova")
    ("Ivan" #:male 43 . "Ivanov")
    ("Anna" #:female 20 . "Karenina")))

(filter (lambda (x) 
  (if (eq? (cadr x) '#:male) #t #f)
) employees)

'(("John" #:male 29 . "Malkovich") ("Ivan" #:male 18 . "Petrov") ("Ivan" #:male 43 . "Ivanov"))

(c) Using higher-order functions (`map`, `ormap`, `andmap`, `filter`, `foldl`)
and without explicit recursion, implement a function `employees-under-21` that
computes a list of full names of employees whose age is under 21 given a list of
employee entries as input:

```lisp
(employees-under-21 employees)
; '(("Ivan" . "Petrov") ("Anna" . "Karenina"))
```


In [52]:
(define employees
  '(("John" #:male 29 . "Malkovich") 
    ("Ivan" #:male 18 . "Petrov")
    ("Anna" #:female 22 . "Petrova")
    ("Ivan" #:male 43 . "Ivanov")
    ("Anna" #:female 20 . "Karenina")))


; defining `third` function for later use
(define (third lst)
    (caddr lst)
)


(define (empoyees-under-21 employees)
    (map (lambda (x)
    (cons (car x) (cdddr x))
    )
     (filter (lambda (x)
      (if (< (third x) 21) #t #f)
    ) employees)
    )
)

(empoyees-under-21 employees)

'(("Ivan" . "Petrov") ("Anna" . "Karenina"))